In [111]:
from alibi.explainers import AnchorTabular
import time
from sklearn.model_selection import  train_test_split
import warnings
warnings.filterwarnings("ignore")
import sys
import os
from sklearn.model_selection import  cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score
sys.path.append(os.path.abspath('/Users/supriyasindigerekumaraswmamy/Desktop/Thesis/wind_Turbine'))
from utils.test_helper import *

In [2]:
import mlflow
mlflow.set_tracking_uri("http://localhost:80") 
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name("predictive_maintenance_system")
    


experiment_id = experiment.experiment_id
runs = client.search_runs(experiment_ids=[experiment_id])
data = []
for run in runs:
        data.append({
            "Run ID": run.data.tags["mlflow.runName"],
            "F1 Score": run.data.metrics.get("score"),
            "Hyperparameters": run.data.params
        }) 
print(data)


[{'Run ID': 'TRANSFORMER', 'F1 Score': 0.9480572308904689, 'Hyperparameters': {'n_estimators': '50', 'max_depth': '80', 'learning_rate': '0.16002527282698065', 'max_features': '20'}}, {'Run ID': 'GENERATOR_BEARING', 'F1 Score': 0.9696340848604224, 'Hyperparameters': {'n_estimators': '100', 'max_depth': '40', 'learning_rate': '0.05738725558581813', 'max_features': '19'}}, {'Run ID': 'HYDRAULIC_GROUP', 'F1 Score': 0.8964187657682071, 'Hyperparameters': {'n_estimators': '200', 'max_depth': '80', 'learning_rate': '0.033460062769701523', 'max_features': '19'}}, {'Run ID': 'GENERATOR', 'F1 Score': 0.9221797301177146, 'Hyperparameters': {'n_estimators': '200', 'max_depth': '10', 'learning_rate': '0.0909594363986538', 'max_features': '20'}}, {'Run ID': 'GEARBOX', 'F1 Score': 0.9348200731743932, 'Hyperparameters': {'n_estimators': '100', 'max_depth': '20', 'learning_rate': '0.17209028508813332', 'max_features': '13'}}]


In [70]:
def get_data():
    failures = load_failures_data('../data/model_data/failures.csv')
    components = failures['component'].unique()
    component_data = load_all_component_data(components)
    data_splits , ui_data_splits= prepare_all_data_for_training(component_data, "target_class")
    models = load_all_models(components, "xgb")
    selected_features_data = fit_and_select_features(models, data_splits)
    models = retrain_models_on_selected_features(models, selected_features_data, data_splits)

    return models, components,selected_features_data,data_splits,ui_data_splits

In [71]:
models,components, selected_features, data_splits ,ui_data= get_data()

In [148]:
def get_data():
    failures = load_failures_data('../data/model_data/failures.csv')
    components = failures['component'].unique()
    component_data = load_all_component_data(components)
    data_splits, ui_data_splits = prepare_all_data_for_training(component_data, "target_class")
    models = load_all_models(components, "xgb")
    selected_features_data = fit_and_select_features(models, data_splits)
    models = retrain_models_on_selected_features(models, selected_features_data, data_splits)
    
    for component in components:
        # Predict and assign the class label
        y_predict = models[component].predict(selected_features_data[component][1])
        selected_features_data[component][1]['target_class'] = y_predict
        selected_features_data[component][1]['target_class'] = selected_features_data[component][1]['target_class']
        
        # Assign turbine_id and convert timestamps
        selected_features_data[component][1]['turbine_id'] = ui_data_splits[component][1]['turbine_id']
        print(type(ui_data_splits[component][1]['turbine_id'] ))
        ui_data_splits[component][1]['timestamp'] = pd.to_datetime(ui_data_splits[component][1]['timestamp'])
        
        
        # Convert to datetime and extract the date
        selected_features_data[component][1]['timestamp'] = ui_data_splits[component][1]['timestamp']
    return selected_features_data, components


In [149]:
selected_features,component = get_data()

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [152]:
#print the instances qithselected_features['GEARBOX'][1]['target_class'] == 1
print(selected_features['GEARBOX'][1][selected_features['GEARBOX'][1]['target_class'] == 1])

      generator_rotations_per_minute_min  \
1131                              1219.4   
229                                  0.0   
495                                  0.0   
1116                                 0.0   
1128                                 0.0   
497                                  0.0   
1078                                 0.0   
522                                  0.0   
802                                  0.0   
1091                                 0.0   
231                                  0.0   
755                                 39.3   
1117                                 0.0   
185                                  0.0   
115                                  0.0   

      generator_bearing_temperature_average  minimum_ambient_wind_speed  \
1131                              64.659722                         1.7   
229                               48.729167                         0.4   
495                               47.840278                         0.

In [49]:
def is_faulty(turbine_id, date_selected, component_data):
    # Convert component data to records for iteration
    for record in component_data.to_dict('records'):
        # Check if turbine_id and date match, and if target_class is 1 (indicating a fault)
        if (record['turbine_id'] == turbine_id and 
            record['timestamp'] == date_selected and 
            record['target_class'] == 1):
            return True
    return False


In [50]:
faulty = is_faulty('T07', '2017-09-05', selected_features['GEARBOX'][1])

In [18]:
turnin_id = selected_features['GEARBOX'][1]['turbine_id']
date = pd.to_datetime(selected_features['GEARBOX'][1]['timestamp'])

In [31]:
date_col = pd.to_datetime(selected_features['GEARBOX'][1]['timestamp'])
date_col = date_col.dt.date

In [13]:
def select_features(models, data_splits):
    selected_features_data = {}
    for component, model in models.items():
        X_train, X_test, y_train, y_test = data_splits[component]
        model.fit(X_train, y_train)
        selector =  model.named_steps['feature_selection']
        selected_features = X_train.columns[selector.get_support()]
        selected_features_train = X_train[selected_features]
        selected_features_train.reset_index(drop=True, inplace=True)
        selected_features_test = X_test[selected_features]
        selected_features_test.reset_index(drop=True, inplace=True)
        selected_features_data[component] = (selected_features_train, selected_features_test, selected_features)
    return selected_features_data

In [14]:
selected_features = select_features(models, data_splits)

In [5]:
for component in components:
    globals()[f"{component}_X_train"] = selected_features[component][0]
    globals()[f"{component}_X_test"] = selected_features[component][1]
    globals()[f"{component}_y_train"] = data_splits[component][2]
    globals()[f"{component}_y_test"] = data_splits[component][3]

In [6]:
for component in components:
    models[component].fit(selected_features[component][0] ,globals()[f"{component}_y_train"])

In [7]:
# confusion matrix
from sklearn.metrics import confusion_matrix

for component in components:
    globals()[f"{component}_y_pred"] = models[component].predict(selected_features[component][1])
    globals()[f"{component}_conf_matrix"] = confusion_matrix(globals()[f"{component}_y_test"], globals()[f"{component}_y_pred"])
    print(globals()[f"{component}_conf_matrix"])

[[679  10]
 [ 36   4]]
[[639  12]
 [ 44  34]]
[[617  17]
 [ 60  35]]
[[688   8]
 [ 18  15]]
[[659  18]
 [ 23  29]]


In [8]:
def train_run(X, y, X_valid, y_valid, model):
    clf = model
    clf.fit(X, y)
    y_valid_pred = clf.predict(X_valid)
    y_train_pred = clf.predict(X)
    f1_train = f1_score(y, y_train_pred, average="weighted")
    precision_train = precision_score(y, y_train_pred, average="weighted")
    recall_train = recall_score(y, y_train_pred, average="weighted")
    f1 = f1_score(y_valid, y_valid_pred, average="weighted")
    precision = precision_score(y_valid, y_valid_pred, average="weighted")
    recall = recall_score(y_valid, y_valid_pred, average="weighted")

    scores = {
        "f1_train": f1_train,
        "precision_train": precision_train,
        "recall_train": recall_train,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

    return scores

In [9]:
for component in components:
    globals()[f"{component}_scores"] = train_run(selected_features[component][0], globals()[f"{component}_y_train"], selected_features[component][1], globals()[f"{component}_y_test"], models[component])
    print(f"{component}_scores: {globals()[f'{component}_scores']}")

GEARBOX_scores: {'f1_train': 1.0, 'precision_train': 1.0, 'recall_train': 1.0, 'f1': 0.9222933495910178, 'precision': 0.9132203872944614, 'recall': 0.9368998628257887}
GENERATOR_scores: {'f1_train': 1.0, 'precision_train': 1.0, 'recall_train': 1.0, 'f1': 0.9141918487192088, 'precision': 0.9145592144368501, 'recall': 0.9231824417009602}
HYDRAULIC_GROUP_scores: {'f1_train': 1.0, 'precision_train': 1.0, 'recall_train': 1.0, 'f1': 0.8806596320227418, 'precision': 0.8803199427549775, 'recall': 0.8943758573388203}
GENERATOR_BEARING_scores: {'f1_train': 1.0, 'precision_train': 1.0, 'recall_train': 1.0, 'f1': 0.9612774980899882, 'precision': 0.9599131641118201, 'recall': 0.9643347050754458}
TRANSFORMER_scores: {'f1_train': 1.0, 'precision_train': 1.0, 'recall_train': 1.0, 'f1': 0.9424417963921411, 'precision': 0.9413631386609856, 'recall': 0.943758573388203}
